# Project 11 : Mobility Hubs in 15 minutes

The **15-Minute City** rides on the concept of **“chrono-urbanism”**, which outlines that the quality of urban life is inversely proportional to the amount of time invested in transportation, more so through the use of automobiles. This concept originated from the first author, Carlos Moreno, who advocates for an urban set-up where locals are able to **access all of their basic essentials at distances that would not take them more than 15 min by foot or by bicycle**. <br>
For the present “15-minute” concept, Moreno supports that residents will be able to enjoy a higher quality of life where they will
be able to effectively fulfil **six essential urban social functions** to sustain a decent urban life.
Those include: 
- living, 
- working, 
- commerce, 
- healthcare, 
- education and 
- entertainment.

Four main dimensions:  
- **density** (people per kilometer square): optimal density that ultimately allows sustainability pursuits to be achieved on the economic, social and environmental frontiers. 
- **proximity** (both spatial and temporal): within the 15-min quickly accessible radial nodes, residents in a given neighborhood can readily access basic services. 
- **diversity**:  twofold: (i) the need for mixed use neighborhoods which are primary in providing a healthy mix of residential, commercial and entertainment components and (ii) diversity in culture and people.
- **digitalization**:  relevant in ensuring the actualization of the three other dimensions. <br><br>

The aim of this project is to propose a significant deployment of the mobility hubs in Louvain so as to ensure a maximum
coverage of the 15-minute measures. <br> 
The idea is to adopt a **Hybrid Deployment Strategy**:
- _Prioritize High-Density Zones_: <br>
Identify zones with higher population density and prioritize the deployment of mobility hubs in these areas. This ensures that a significant portion of the population benefits from reduced travel times.

- _Maximize Efficiency in Strategic Zones_: <br>
Within high-density zones, focus on strategic areas where essential functions (living, working, commerce, health care, education, and entertainment) converge. Deploy additional hubs to minimize travel time for residents accessing these crucial services.

- _Average Travel Time Optimization_: <br>
Extend the deployment to other zones, aiming for an average travel time across the city that is below the 15-minute threshold. This involves sacrificing travel time in some zones to enhance overall accessibility.


In [6]:
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point
import pandas as pd
import numpy as np 

In [4]:
# reading a file with the sectors (identified by parameter CODSEC) which has been officially used when collecting socio-economic data
zone = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/leuven_statsec.gpkg')
zone = zone.to_crs('epsg:4326')
zone.reset_index(inplace=True)

zone.head(3)

,index,UIDN,OIDN,CODSEC,NISCODE,SEC,SECNAAM,LENGTE,OPPERVL,STDEEL,geometry
0,0,19880,4074,24062A61-,24062,A61-,KAREELVELD,5896.98,1529378.12,LEUVEN NOORD,"MULTIPOLYGON (((4.68850 50.89458, 4.68895 50.8..."
1,1,20055,4078,24062B100,24062,B100,PUTKAPEL-CENTRUM,5277.17,980502.76,WILSELE WIJGMAAL,"MULTIPOLYGON (((4.72796 50.93378, 4.72742 50.9..."
2,2,20056,4087,24062B34-,24062,B34-,ROESELBERG,4205.81,643689.23,WILSELE WIJGMAAL,"MULTIPOLYGON (((4.69867 50.89670, 4.69866 50.8..."


In [ ]:
# a folium map depicting the tessellation of Leuven based on socio-demographic zones 

# customised function for changing style in folium maps 
def style(fill_color, border_color, spessore_contorno=2, opacita=0.5):
    return {
        'fillColor': fill_color,
        'color': border_color,
        'weight': spessore_contorno,
        'fillOpacity': opacita
    }


m = folium.Map(location = (zone.unary_union.centroid.y, zone.unary_union.centroid.x), zoom_start = 12.5) # zone.unary_union.centroid punto centrale di tutta la città 
folium.GeoJson(zone, style_function = lambda x: style('purple', 'white', spessore_contorno = 0.6, opacita = 0.6)).add_to(m)

In [235]:
hubs = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/hub_data_leuven/mobility_hubs.gpkg', crs={'init':'epsg:31370'})
hubs = hubs.to_crs(epsg=4326)

column_name_translations = {  #translating into english 
    'X': 'Longitude',
    'Y': 'Latitude',
    'nr': 'Number',
    'naam punt': 'Point Name',
    'Deelgemeente': 'Sub-municipality',
    'openbaar vervoer': 'Public Transport',
    'AWV': 'Roads and Traffic Agency',
    'OD of andere': 'Other',
    'Herkoms': 'Origin',
    'Interregionaal': 'Interregional',
    'Tijdstip lancering eHUB': 'eHUB Launch Time',
    'realistische timing': 'Realistic Timing',
    'publieke fietsenstalling': 'Public Bike Storage',
    'publieke parking': 'Public Parking',
    'Pakjesautomaat': 'Parcel Machine',
    'Blue Bike': 'Blue Bike',
    'aantal deelwagens': 'Number of Shared Cars',
    'aantal laadpalen': 'Number of Charging Stations',
    'aantal e-deelwagens': 'Number of Electric Shared Cars',
    'type zuil': 'Pillar Type',
    'aantal deelwagens (Uitbr 2020)': 'Number of Shared Cars (Expansion in 2020)',
    'aantal e-deelwagens (uitbr 2020)': 'Number of Electric Shared Cars (Expansion in 2020)',
    'aantal laadpalen (dossier 2020 - voorzien voor Q1 2021)': 'Number of Charging Stations (File 2020 - Planned for Q1 2021)',
    'aantal deelwagens (scenario2)': 'Number of Shared Cars (Scenario 2)',
    'aantal e-deelwagens (scenario2)': 'Number of Electric Shared Cars (Scenario 2)',
    'e-deelfietsen Sept (30)': 'E-bikes in September (30)',
    'e-deelfietsen Maart (35)': 'E-bikes in March (35)',
    'e-deelfietsen Mei (40)': 'E-bikes in May (40)',
    'e-deelfietsen Toekomst? (60)': 'E-bikes in the Future? (60)',
    'aantal deelfietsen': 'Number of Shared Bikes',
    'aantal e-deelfietsen (indien max 92 0)': 'Number of Electric Shared Bikes (if max 92 0)',
    'aantal e-deelfietsen (indien max 125)': 'Number of Electric Shared Bikes (if max 125)',
    'aantal e-deelfietsen (indien max 125) scenario 26 locaties': 'Number of Electric Shared Bikes (if max 125) scenario 26 locations',
    'aantal e-deelfietsen (indien max 125) scenario 31 locaties': 'Number of Electric Shared Bikes (if max 125) scenario 31 locations',
    'aantal e-deelbakfietsen': 'Number of Electric Cargo Bikes',
    'aantal pakjesautomaten': 'Number of Parcel Machines',
    '# fietsbeugels te voorzien': 'Number of Bike Racks to be Provided',
    'aantal e-hubs': 'Number of E-hubs',
    'EV capaciteit': 'Electric Vehicle Capacity',
    'stadsgezicht': 'Cityscape',
    'geometry': 'geometry'
}

hubs = hubs.rename(columns=column_name_translations)

for index, hub in hubs.iterrows():
    lon, lat = hub['Longitude'], hub['Latitude']

    feature1 = hub['Point Name']
    feature2 = f"Number of Shared Cars: {hub['Number']}"
    feature3 = f"Public Transport: {hub['Public Transport']}"
    feature4 = f"Number of Shared Bikes: {hub['Number of Shared Bikes']}"
    feature5 = f"Number of Electric Shared Cars: {hub['Number of Electric Shared Cars']}"
    feature6 = f"Number of Charging Stations: {hub['Number of Charging Stations']}"

    popup_text = f"<strong>{feature1}</strong><br>{feature2}<br>{feature3}<br>{feature4}<br>{feature5}<br>{feature6}"


    marker = folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color='darkpurple',icon = 'bell') # icon = user or star or bell or flag
    )


    marker.add_to(m)
m

## ISOCHRONE APPROACH: how far can we travel in 15 minutes? 

## Attempt using mapboxOSRM

In [ ]:
from pyproj import Proj, transform 

hubs_copia = hubs.copy()
hubs_copia.crs = 'EPSG:4326'
central_point_coords = zone.unary_union.centroid.x, zone.unary_union.centroid.y
central_point = Point(central_point_coords)

# converto il punto centrale nello stesso epsg 23029 che è quello del belgio (da epsg.io)
projector = Proj(init='epsg:4326'), Proj(init='epsg:23029')
central_point_x, central_point_y = transform(projector[0], projector[1], central_point_coords[0], central_point_coords[1])

central_point_df = gpd.GeoDataFrame(geometry=[Point(central_point_x, central_point_y)], crs='EPSG:23029')
hubs_copia = hubs_copia.to_crs('EPSG:23029')
hubs_copia['distance_to_central'] = hubs_copia.distance(central_point_df['geometry'].iloc[0])

closest_hub = hubs_copia.loc[hubs_copia['distance_to_central'].idxmin()]
closest_hub_df = gpd.GeoDataFrame([closest_hub], geometry='geometry', crs='EPSG:23029')
closest_hub_df.to_crs('EPSG:4326')


In [421]:
from routingpy.routers import MapboxOSRM

MY_MAPBOXOSRM_API_KEY = 'pk.eyJ1IjoidnBpYyIsImEiOiJjbHI3aXQ5Mm0yOWlkMmpudnBtNHJ5OGx0In0.yXnhEOyVnjz6nriA4GK2-g'
mb = MapboxOSRM(api_key = MY_MAPBOXOSRM_API_KEY)

def mb_isochrone(gdf, time, profile = "driving"):

    gdf['LON_VALUE'] = gdf.to_crs(4326).geometry.x
    gdf['LAT_VALUE'] = gdf.to_crs(4326).geometry.y

    coordinates = gdf[['LON_VALUE', 'LAT_VALUE']].values.tolist()

    isochrone_shapes = []

    if type(time) is not list:
        time = [time]

    time_seconds = [60 * x for x in time]

    # Given the way that routingpy works, we need to iterate through the list of 
    # coordinate pairs, then iterate through the object returned and extract the 
    # isochrone geometries.  
    for c in coordinates:
        iso_request = mb.isochrones(locations = c, profile = profile,
                                    intervals = time_seconds, polygons = "true")
        #print('iso request:', iso_request)
        for i in iso_request:
            iso_geom = Polygon(i.geometry[0])
            isochrone_shapes.append(iso_geom)

    # re-build the dataset but with isochrone geometries
    df_values = gdf.drop(columns = ['geometry', 'LON_VALUE', 'LAT_VALUE'])

    time_col = time * len(df_values)

    # need to repeat the dataframe to account for multiple time intervals
    df_values_rep = pd.DataFrame(np.repeat(df_values.values, len(time_seconds), axis = 0))
    df_values_rep.columns = df_values.columns

    isochrone_gdf = gpd.GeoDataFrame(
        data = df_values_rep,
        geometry = isochrone_shapes,
        crs = 4326
    )

    isochrone_gdf['time'] = time_col

    # sorting the dataframe in descending order of time to improve visualization
    # (the smallest isochrones should go on top, which means they are plotted last)
    isochrone_gdf = isochrone_gdf.sort_values('time', ascending = False)

    return(isochrone_gdf)

In [358]:
isocrone_walking = mb_isochrone(closest_hub_df, time = [5, 10,15], 
                            profile = "walking")

n = folium.Map(location = (central_point_coords[1], central_point_coords[0]), zoom_start = 12.5) 
folium.GeoJson(isocrone_walking, style_function = lambda x: style('#6f00ff', 'black', spessore_contorno = 1, opacita = 0.4)).add_to(n)
folium.Marker(
        location=[float(closest_hub_df.loc[:,'LAT_VALUE']), float(closest_hub_df.loc[:, 'LON_VALUE'])],
        icon=folium.Icon(color='darkpurple', icon = 'user') # icon = user or star or bell or flag
    ).add_to(n)


Isochrones([Isochrone([[[4.705406, 50.886084], [4.702904, 50.885926], [4.70072, 50.884424], [4.700137, 50.883424], [4.700764, 50.882424], [4.700389, 50.880407], [4.701734, 50.879753], [4.704406, 50.880032], [4.705406, 50.879259], [4.707406, 50.879166], [4.708406, 50.880387], [4.709686, 50.880424], [4.710616, 50.882424], [4.710652, 50.884424], [4.708406, 50.885313], [4.707406, 50.884868], [4.706406, 50.885169], [4.705406, 50.886084]]], 300), Isochrone([[[4.707406, 50.889641], [4.706406, 50.889688], [4.703406, 50.888163], [4.700406, 50.888229], [4.699406, 50.887847], [4.695529, 50.884424], [4.696128, 50.883424], [4.696159, 50.879424], [4.698406, 50.876949], [4.700406, 50.876805], [4.705264, 50.877566], [4.706406, 50.876601], [4.710406, 50.877094], [4.711779, 50.878051], [4.713979, 50.878424], [4.715035, 50.882424], [4.713406, 50.883891], [4.711786, 50.886804], [4.710406, 50.886696], [4.709406, 50.887645], [4.707906, 50.887924], [4.707406, 50.889641]]], 600), Isochrone([[[4.707406, 50.893

In [352]:
isocrone_bike = mb_isochrone(closest_hub_df, time = [5, 10,15], 
                            profile = "cycling")

bike_map = folium.Map(location = (central_point_coords[1], central_point_coords[0]), zoom_start = 12.5) 
folium.GeoJson(isocrone_bike, style_function = lambda x: style('#6f00ff', 'black', spessore_contorno = 1, opacita = 0.4)).add_to(bike_map)
folium.Marker(
        location=[float(closest_hub_df.loc[:,'LAT_VALUE']), float(closest_hub_df.loc[:, 'LON_VALUE'])],
        icon=folium.Icon(color='darkpurple', icon = 'user') # icon = user or star or bell or flag
    ).add_to(bike_map)


In [353]:
from IPython.display import display, HTML

n.get_root().html.add_child(folium.Element("""
                                    <h3 align="center" style="font-size:16px"><b>Walking isochrones starting from an existing mobility hub</b></h3>
                                    """)
                                    )
bike_map.get_root().html.add_child(folium.Element("""
                                    <h3 align="center" style="font-size:16px"><b>Cycling isochrones starting from an existing mobility hub</b></h3>
                                    """)
                                    )

display(HTML(f'<div style="display:flex">{n._repr_html_()} {bike_map._repr_html_()}</div>'))

# a sinistra: partendo dall'icona centrale, dove arrivo camminando in 5, 10 e 15 minuti
# a destra: partendo dall'icona centrale, dove arrivo in bici in 5, 10 e 15 minuti

In [ ]:
import folium
from folium.plugins import MeasureControl, MiniMap

def create_isochrone_layer(profile, time, color, icon):
    isochrone = mb_isochrone(closest_hub_df, time=time, profile=profile)
    
    layer = folium.FeatureGroup(name=f'Isochrones for {profile.capitalize()}')
    folium.GeoJson(isochrone, style_function=lambda x: style(color, 'black', weight=1, opacity=0.4)).add_to(layer)
    
    folium.Marker(
        location=[float(closest_hub_df.loc[:, 'LAT_VALUE']), float(closest_hub_df.loc[:, 'LON_VALUE'])],
        icon=folium.Icon(color='darkpurple', icon=icon)  # icon = user or star or bell or flag
    ).add_to(layer)
    
    return layer

main_map = folium.Map(location = (central_point_coords[1], central_point_coords[0]), tiles = 'cartodbdark_matter',zoom_start=12.5)

# isochrone layers for walking and cycling
walking_layer = create_isochrone_layer(profile='walking', time=[5, 10, 15], color='blue')
cycling_layer = create_isochrone_layer(profile='cycling', time=[5, 10, 15], color='yellow')

main_map.add_child(cycling_layer)
main_map.add_child(walking_layer)

MeasureControl(primary_length_unit='kilometers').add_to(main_map)
MiniMap(toggle_display=True).add_to(main_map)
folium.LayerControl().add_to(main_map)

display(main_map)


## Algoritmo

In [ ]:
# voglio tutti i punti raggiungibili a partire da: 
# a.  centroidi della zona 
# b.  dalla stazione centrale di Leuven e poi andando avanti con gli altri punti precedentemente trovati --> sort of hub and spoke model 

In [429]:
# stazione centrale di Leuven coordinate lon = 4.714594 lat = 50.882175 (https://epsg.io/map#srs=4326&x=4.714594&y=50.882175&z=16&layer=streets)
station = (4.714594, 50.882175)

start_time_intervals = [5, 10, 15]  
max_hubs = 10  

hub_locations = [station]
geometry = [Point(station)]
gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(geometry), crs="EPSG:4326")

isochrone_data = {}

for hub_location in hub_locations:
    for time_interval in start_time_intervals:
        isochrone_data[(hub_location, time_interval)] = mb_isochrone(gdf, time_interval, profile="cycling")

# prune and select final hub candidates
final_hub_locations = []

'''
for time_interval in start_time_intervals:
    # Concatenate isochrone data for all hub locations at the current time interval
    concatenated_isochrones = pd.concat([isochrone_data[(hub_location, time_interval)] for hub_location in hub_locations])

    # Prune and select the top locations with the highest coverage
    top_hubs = concatenated_isochrones.nlargest(max_hubs, 'time')
    final_hub_locations.extend(top_hubs[['LAT_VALUE', 'LON_VALUE']].values.tolist())

# Step 7: Visualize the result on a map
hub_map = folium.Map(location=central_point_coords, zoom_start=13)
for location in final_hub_locations:
    folium.Marker(location, icon=folium.Icon(color='darkpurple', icon='star')).add_to(hub_map)

# Display the map
hub_map'''

"\nfor time_interval in start_time_intervals:\n    # Concatenate isochrone data for all hub locations at the current time interval\n    concatenated_isochrones = pd.concat([isochrone_data[(hub_location, time_interval)] for hub_location in hub_locations])\n\n    # Prune and select the top locations with the highest coverage\n    top_hubs = concatenated_isochrones.nlargest(max_hubs, 'time')\n    final_hub_locations.extend(top_hubs[['LAT_VALUE', 'LON_VALUE']].values.tolist())\n\n# Step 7: Visualize the result on a map\nhub_map = folium.Map(location=central_point_coords, zoom_start=13)\nfor location in final_hub_locations:\n    folium.Marker(location, icon=folium.Icon(color='darkpurple', icon='star')).add_to(hub_map)\n\n# Display the map\nhub_map"

In [432]:
# plotta le isocrone trovate (per ora a partire solo dalla stazione) - 5, 10, 15 minuti a piedi
gdf_list = [gdff.reset_index(drop=True) for gdff in isochrone_data.values()]
gdf_isoc = gpd.GeoDataFrame(pd.concat(gdf_list, keys=isochrone_data.keys(), names=['geo', 'tempi']))
gdf_isoc.reset_index(inplace=True)

prova_map = folium.Map(location = (central_point_coords[1], central_point_coords[0]), zoom_start = 12.5) 
folium.GeoJson(gdf_isoc, style_function = lambda x: style('#6f00ff', 'black', spessore_contorno = 1, opacita = 0.4)).add_to(prova_map)

folium.Marker(location=[central_point_coords[1], central_point_coords[0]], icon=folium.Icon(color='black', icon='user')).add_to(prova_map)


# aggiungo anche il boundary più esterno --> perimetro di referenza per l'analisi 
boundary = zone.unary_union
boundary_gdf = gpd.GeoDataFrame(geometry=[boundary], crs="EPSG:4326")
folium.GeoJson(boundary_gdf, style_function = lambda x: style('purple', 'purple', spessore_contorno = 3, opacita = 0.01)).add_to(prova_map)
prova_map 


In [433]:
# l'isocrona output della chiamata api a mapbox è un poligono che descrive lo spazio che è percorribile in k minuti
# siamo interessati ai punti che compongono il poligono perchè da questi punti è come se facessimo partire un altro modello hub&spoke 
# per semplicità consideriamo l'attributo exterior, che permette di accedere all'outer ring del poligono e le coordinate dei suoi punti 

coordinates_list = list(gdf_isoc['geometry'][2].exterior.coords) # exterior gives access to the outer ring of the polygon, and coords returns the coordinates of that ring as a sequence of tuples
print(len(coordinates_list))
print(coordinates_list)

# aggiungo questi punti alla mappa (candidate hubs), sono i black dots 
for coord in coordinates_list:
    folium.CircleMarker(location=[coord[1], coord[0]], radius=3, color='black', fill=True, fill_color='black').add_to(prova_map)

prova_map

321
[(4.716594, 50.91647), (4.715594, 50.915708), (4.714594, 50.916256), (4.713537, 50.915175), (4.715012, 50.914593), (4.714912, 50.913857), (4.712594, 50.913471), (4.711176, 50.913757), (4.71023, 50.915175), (4.708888, 50.915469), (4.708594, 50.915898), (4.708424, 50.915175), (4.709191, 50.914175), (4.708306, 50.913175), (4.707594, 50.910752), (4.706902, 50.913483), (4.704594, 50.916119), (4.704234, 50.914175), (4.704948, 50.913529), (4.704905, 50.910175), (4.705786, 50.909367), (4.705833, 50.908175), (4.704095, 50.907674), (4.703846, 50.907175), (4.70475, 50.906331), (4.704783, 50.905175), (4.703961, 50.904808), (4.703594, 50.903009), (4.703237, 50.903818), (4.700374, 50.904175), (4.701084, 50.904685), (4.701318, 50.905899), (4.700313, 50.907175), (4.700219, 50.913175), (4.69895, 50.913531), (4.698594, 50.914108), (4.69839, 50.913175), (4.699177, 50.912175), (4.698762, 50.906007), (4.696398, 50.905979), (4.695271, 50.907175), (4.695015, 50.908596), (4.694132, 50.909175), (4.693768, 